## Week 1
### Data Capture from Binance


In [1]:
pip install websockets pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncio
import websockets
import json
import pandas as pd
import datetime
import os
import glob
import numpy as np
import ast 

In [8]:
buffer = []

async def binanceDepthCollector():
    dep_url = f"wss://stream.binance.com:9443/ws/btcusdt@depth10@100ms"
    async with websockets.connect(dep_url) as websocket:
        print('Connected')
        while True:
            try:
                info = await websocket.recv()
                #print('start listenning') #测一下
                data = json.loads(info)
                now = datetime.datetime.now()
                timestamp = now.strftime('%Y-%m-%d %H:%M:%S.%f')

                rows = []#存储当前访问的depth

                #print(data)#测一下读取到的数据格式

                for i, (bid,ask) in enumerate(zip(data['bids'],data['asks'])):
                    price1, qty1 = bid
                    price2, qty2 = ask
                    rows.append([timestamp,data['lastUpdateId'],[float(price1),float(qty1)],[float(price2),float(qty2)],i+1])

                buffer.extend(rows)#准备该次访问写入的数据
                
                df = pd.DataFrame(buffer, columns = ['timestamp','lastUpdateId','bids','asks','level'])
                filename = "depth10_btcusdt_live_7_10_2.csv"
                header_adj = not os.path.exists(filename)
                df.to_csv(filename, mode = 'a', header = header_adj, index = False)
                #print('file saved')

                buffer.clear()#清空缓存

            except Exception as e:
                print(f"[Error] {e}")
                await asyncio.sleep(5)  

In [9]:
buffer.clear()#清空缓存
await binanceDepthCollector()

Connected


CancelledError: 

In [ ]:
await websocket.close()

In [4]:
import requests

#这里批量下载一下5m级别k线数据
symbol = "BTCUSDT"
interval = "5m"
start_date = datetime.datetime(2025, 1, 1)
end_date = datetime.datetime(2025, 7, 12)
base_url = "https://data.binance.vision/data/spot/daily/klines"

date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m-%d")
    filename = f"{symbol}-{interval}-{date_str}.zip"
    url = f"{base_url}/{symbol}/{interval}/{filename}"
    
    try:
        print(f"Downloading {filename}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Saved: {filename}")
        else:
            print(f"Failed: {filename} (status code {response.status_code})")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")
    
    date += datetime.timedelta(days=1)


Saved: BTCUSDT-5m-2025-01-01.zip
Saved: BTCUSDT-5m-2025-01-02.zip
Saved: BTCUSDT-5m-2025-01-03.zip
Saved: BTCUSDT-5m-2025-01-04.zip
Saved: BTCUSDT-5m-2025-01-05.zip
Saved: BTCUSDT-5m-2025-01-06.zip
Saved: BTCUSDT-5m-2025-01-07.zip
Saved: BTCUSDT-5m-2025-01-08.zip
Saved: BTCUSDT-5m-2025-01-09.zip
Saved: BTCUSDT-5m-2025-01-10.zip
Saved: BTCUSDT-5m-2025-01-11.zip
Saved: BTCUSDT-5m-2025-01-12.zip
Saved: BTCUSDT-5m-2025-01-13.zip
Saved: BTCUSDT-5m-2025-01-14.zip
Saved: BTCUSDT-5m-2025-01-15.zip
Saved: BTCUSDT-5m-2025-01-16.zip
Saved: BTCUSDT-5m-2025-01-17.zip
Saved: BTCUSDT-5m-2025-01-18.zip
Saved: BTCUSDT-5m-2025-01-19.zip
Saved: BTCUSDT-5m-2025-01-20.zip
Saved: BTCUSDT-5m-2025-01-21.zip
Saved: BTCUSDT-5m-2025-01-22.zip
Saved: BTCUSDT-5m-2025-01-23.zip
Saved: BTCUSDT-5m-2025-01-24.zip
Saved: BTCUSDT-5m-2025-01-25.zip
Saved: BTCUSDT-5m-2025-01-26.zip
Saved: BTCUSDT-5m-2025-01-27.zip
Saved: BTCUSDT-5m-2025-01-28.zip
Saved: BTCUSDT-5m-2025-01-29.zip
Saved: BTCUSDT-5m-2025-01-30.zip
Saved: BTC

In [5]:
import zipfile
# 设置文件夹路径（当前目录下所有 .zip 文件）
zip_folder = "./5mkline"  

# 遍历当前目录所有 zip 文件
for filename in os.listdir(zip_folder):
    if filename.endswith(".zip"):
        zip_path = os.path.join(zip_folder, filename)
        extract_dir = os.path.join(zip_folder, filename.replace(".zip", ""))
        
        print(f"Extracting {filename} to {extract_dir} ...")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
        except zipfile.BadZipFile:
            print(f"Bad zip file: {filename}")

Extracting BTCUSDT-5m-2025-01-01.zip to ./5mkline\BTCUSDT-5m-2025-01-01 ...
Extracting BTCUSDT-5m-2025-01-02.zip to ./5mkline\BTCUSDT-5m-2025-01-02 ...
Extracting BTCUSDT-5m-2025-01-03.zip to ./5mkline\BTCUSDT-5m-2025-01-03 ...
Extracting BTCUSDT-5m-2025-01-04.zip to ./5mkline\BTCUSDT-5m-2025-01-04 ...
Extracting BTCUSDT-5m-2025-01-05.zip to ./5mkline\BTCUSDT-5m-2025-01-05 ...
Extracting BTCUSDT-5m-2025-01-06.zip to ./5mkline\BTCUSDT-5m-2025-01-06 ...
Extracting BTCUSDT-5m-2025-01-07.zip to ./5mkline\BTCUSDT-5m-2025-01-07 ...
Extracting BTCUSDT-5m-2025-01-08.zip to ./5mkline\BTCUSDT-5m-2025-01-08 ...
Extracting BTCUSDT-5m-2025-01-09.zip to ./5mkline\BTCUSDT-5m-2025-01-09 ...
Extracting BTCUSDT-5m-2025-01-10.zip to ./5mkline\BTCUSDT-5m-2025-01-10 ...
Extracting BTCUSDT-5m-2025-01-11.zip to ./5mkline\BTCUSDT-5m-2025-01-11 ...
Extracting BTCUSDT-5m-2025-01-12.zip to ./5mkline\BTCUSDT-5m-2025-01-12 ...
Extracting BTCUSDT-5m-2025-01-13.zip to ./5mkline\BTCUSDT-5m-2025-01-13 ...
Extracting B

In [2]:
zip_folder = "./5mkline"  
parent_dir = zip_folder

# 匹配所有子文件夹里的 CSV 文件（递归找）
csv_files = glob.glob(os.path.join(parent_dir, "BTCUSDT-5m-*", "*.csv"))

print(f"找到 {len(csv_files)} 个 CSV 文件")

# 读取并合并所有 CSV
df_5mkline = pd.concat([pd.read_csv(f, header=None) for f in csv_files], ignore_index=True)

# Binance CSV 列名
df_5mkline.columns = [
    'timestamp', 'open', 'high', 'low', 'close', 'volume',
    'close_time', 'quote_asset_volume', 'number_of_trades',
    'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'
]


# 查看结果
print(df_5mkline.head())
print(f"总共 {len(df_5mkline)} 行数据")

找到 193 个 CSV 文件
          timestamp      open      high       low     close    volume  \
0  1735689600000000  93576.00  93702.15  93537.50  93661.20  45.94160   
1  1735689900000000  93661.20  93678.02  93600.00  93607.10  71.94585   
2  1735690200000000  93607.10  93656.19  93489.03  93656.18  57.96928   
3  1735690500000000  93656.19  93840.05  93614.95  93796.35  39.74456   
4  1735690800000000  93796.35  93898.05  93707.81  93740.00  38.38053   

         close_time  quote_asset_volume  number_of_trades  \
0  1735689899999999        4.301907e+06              7448   
1  1735690199999999        6.736121e+06              4165   
2  1735690499999999        5.423766e+06              8175   
3  1735690799999999        3.727084e+06              8031   
4  1735691099999999        3.600685e+06             10064   

   taker_buy_base_volume  taker_buy_quote_volume  ignore  
0               16.87967            1.580504e+06       0  
1               16.66761            1.560680e+06       0  
2

In [3]:
df_depth = pd.read_csv('depth10_btcusdt_live_7_10_1.csv', header = 0)
print(df_depth.head())
print(f"总共 {len(df_depth)} 行数据")

                    timestamp  lastUpdateId                  bids  \
0  2025-07-10 16:20:42.758256   72513320823  [111145.88, 0.49071]   
1  2025-07-10 16:20:42.758256   72513320823  [111145.87, 0.00081]   
2  2025-07-10 16:20:42.758256   72513320823  [111145.85, 0.00021]   
3  2025-07-10 16:20:42.758256   72513320823  [111145.74, 0.00032]   
4  2025-07-10 16:20:42.758256   72513320823  [111145.73, 0.32393]   

                    asks  level  
0  [111145.89, 11.09448]      1  
1    [111145.9, 0.00035]      2  
2    [111145.91, 0.0001]      3  
3    [111145.92, 0.2158]      4  
4   [111145.93, 0.00035]      5  
总共 24980660 行数据


In [4]:
df_depth[0:11]

,timestamp,lastUpdateId,bids,asks,level
0,2025-07-10 16:20:42.758256,72513320823,"[111145.88, 0.49071]","[111145.89, 11.09448]",1
1,2025-07-10 16:20:42.758256,72513320823,"[111145.87, 0.00081]","[111145.9, 0.00035]",2
2,2025-07-10 16:20:42.758256,72513320823,"[111145.85, 0.00021]","[111145.91, 0.0001]",3
3,2025-07-10 16:20:42.758256,72513320823,"[111145.74, 0.00032]","[111145.92, 0.2158]",4
4,2025-07-10 16:20:42.758256,72513320823,"[111145.73, 0.32393]","[111145.93, 0.00035]",5
5,2025-07-10 16:20:42.758256,72513320823,"[111145.12, 0.00134]","[111146.51, 0.00134]",6
6,2025-07-10 16:20:42.758256,72513320823,"[111145.0, 0.001]","[111146.77, 0.01817]",7
7,2025-07-10 16:20:42.758256,72513320823,"[111144.82, 5e-05]","[111147.54, 0.0001]",8
8,2025-07-10 16:20:42.758256,72513320823,"[111144.8, 0.00018]","[111147.55, 0.07137]",9
9,2025-07-10 16:20:42.758256,72513320823,"[111144.55, 5e-05]","[111147.61, 0.0001]",10


In [18]:
interval_2 = "1h"
zip_folder_2 = "./1hkline" 

os.makedirs(zip_folder_2, exist_ok=True)

date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m-%d")
    filename = f"{symbol}-{interval_2}-{date_str}.zip"
    url = f"{base_url}/{symbol}/{interval_2}/{filename}"
    save_path = os.path.join(zip_folder_2, filename)
    
    try:
        print(f"Downloading {filename}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            #print(f"Saved: {filename}")
        else:
            print(f"Failed: {filename} (status code {response.status_code})")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")
    
    date += datetime.timedelta(days=1)
 

# 遍历当前目录所有 zip 文件
for filename in os.listdir(zip_folder_2):
    if filename.endswith(".zip"):
        zip_path = os.path.join(zip_folder_2, filename)
        extract_dir = os.path.join(zip_folder_2, filename.replace(".zip", ""))
        
        print(f"Extracting {filename} to {extract_dir} ...")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
        except zipfile.BadZipFile:
            print(f"Bad zip file: {filename}")

Extracting BTCUSDT-1h-2025-01-01.zip to ./1hkline\BTCUSDT-1h-2025-01-01 ...
Extracting BTCUSDT-1h-2025-01-02.zip to ./1hkline\BTCUSDT-1h-2025-01-02 ...
Extracting BTCUSDT-1h-2025-01-03.zip to ./1hkline\BTCUSDT-1h-2025-01-03 ...
Extracting BTCUSDT-1h-2025-01-04.zip to ./1hkline\BTCUSDT-1h-2025-01-04 ...
Extracting BTCUSDT-1h-2025-01-05.zip to ./1hkline\BTCUSDT-1h-2025-01-05 ...
Extracting BTCUSDT-1h-2025-01-06.zip to ./1hkline\BTCUSDT-1h-2025-01-06 ...
Extracting BTCUSDT-1h-2025-01-07.zip to ./1hkline\BTCUSDT-1h-2025-01-07 ...
Extracting BTCUSDT-1h-2025-01-08.zip to ./1hkline\BTCUSDT-1h-2025-01-08 ...
Extracting BTCUSDT-1h-2025-01-09.zip to ./1hkline\BTCUSDT-1h-2025-01-09 ...
Extracting BTCUSDT-1h-2025-01-10.zip to ./1hkline\BTCUSDT-1h-2025-01-10 ...
Extracting BTCUSDT-1h-2025-01-11.zip to ./1hkline\BTCUSDT-1h-2025-01-11 ...
Extracting BTCUSDT-1h-2025-01-12.zip to ./1hkline\BTCUSDT-1h-2025-01-12 ...
Extracting BTCUSDT-1h-2025-01-13.zip to ./1hkline\BTCUSDT-1h-2025-01-13 ...
Extracting B

In [5]:
zip_folder_2 = "./1hkline" 
parent_dir_1h = zip_folder_2

# 匹配所有子文件夹里的 CSV 文件（递归找）
csv_files_1h = glob.glob(os.path.join(parent_dir_1h, "BTCUSDT-1h-*", "*.csv"))

print(f"找到 {len(csv_files_1h)} 个 CSV 文件")

# 读取
df_1hkline = pd.concat([pd.read_csv(f, header=None) for f in csv_files_1h], ignore_index=True)

df_1hkline.columns = [
    'timestamp', 'open', 'high', 'low', 'close', 'volume',
    'close_time', 'quote_asset_volume', 'number_of_trades',
    'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'
]

#检查
print(df_1hkline.head())
print(f"总共 {len(df_1hkline)} 行数据")

找到 193 个 CSV 文件
          timestamp      open      high       low     close     volume  \
0  1735689600000000  93576.00  94509.42  93489.03  94401.14  755.99010   
1  1735693200000000  94401.13  94408.72  93578.77  93607.74  586.53456   
2  1735696800000000  93607.74  94105.12  93594.56  94098.91  276.78045   
3  1735700400000000  94098.90  94098.91  93728.22  93838.04  220.99302   
4  1735704000000000  93838.04  93838.04  93500.00  93553.91  279.46909   

         close_time  quote_asset_volume  number_of_trades  \
0  1735693199999999        7.106881e+07             93525   
1  1735696799999999        5.509661e+07             79943   
2  1735700399999999        2.597409e+07             55078   
3  1735703999999999        2.074804e+07             35001   
4  1735707599999999        2.617906e+07             38597   

   taker_buy_base_volume  taker_buy_quote_volume  ignore  
0              421.08319            3.959678e+07       0  
1              257.42023            2.418794e+07      

In [6]:
df = df_depth.copy()
df['bids'] = df['bids'].apply(ast.literal_eval)
df['asks'] = df['asks'].apply(ast.literal_eval)
grouped = df.groupby('timestamp').agg({
    'bids': lambda x: np.array(list(x)),
    'asks': lambda x: np.array(list(x)),
    'lastUpdateId': 'first'
}).reset_index()

In [7]:
high_freq = []
for i, row in grouped.iterrows():
    timestamp = row['timestamp']
    bids = row['bids'][:10]
    asks = row['asks'][:10]

    bids_price = [float(b[0]) for b in bids]
    bids_qty = [float(b[1]) for b in bids]

    ask_price = [float(a[0]) for a in asks]
    ask_qty = [float(a[1]) for a in asks]
    
    spread = ask_price[0] - bids_price[0]
    mid_price = (ask_price[0] + bids_price[0])/2
    obi = (bids_qty[0] - ask_qty[0])/(bids_qty[0] + ask_qty[0] +1e-6)
    micro_price = (ask_price[0]*ask_qty[0]+bids_price[0]*bids_qty[0])/(ask_qty[0]+bids_qty[0])

    high_freq.append({
        'timestamp': timestamp,
        'spread': spread,
        'micro_price': micro_price,
        'imbalance': obi,
    })

features_df = pd.DataFrame(high_freq)
    




In [ ]:
def compute_high_freq_features(df_100ms):
    # 假设包含 bid_1, ask_1, vol_bid_1, vol_ask_1 等字段
    df = df_100ms.copy()
    df['mid_price'] = (df['bid_1'] + df['ask_1']) / 2
    df['spread'] = df['ask_1'] - df['bid_1']
    df['obi'] = (df['vol_bid_1'] - df['vol_ask_1']) / (df['vol_bid_1'] + df['vol_ask_1'] + 1e-6)
    df['micro_price'] = (df['ask_1'] * df['vol_bid_1'] + df['bid_1'] * df['vol_ask_1']) / (df['vol_bid_1'] + df['vol_ask_1'] + 1e-6)
    
    # Order Flow Imbalance（你可以扩展更复杂版本）
    df['ofi'] = df['vol_bid_1'].diff().fillna(0) - df['vol_ask_1'].diff().fillna(0)

    # 滑动窗口特征
    win = 30  # 对应3秒（100ms * 30）
    df['spread_mean_3s'] = df['spread'].rolling(win).mean()
    df['spread_std_3s'] = df['spread'].rolling(win).std()
    df['depth_change_rate'] = (df['vol_bid_1'] + df['vol_ask_1']).pct_change(win)
    
    return df

def compute_mid_freq_features(df_kline_5min):
    df = df_kline_5min.copy()
    df['ma_5min'] = df['close'].rolling(3).mean()
    df['mid_return_5min'] = np.log(df['close'] / df['close'].shift(1))

    # Bollinger Band Width
    mid = df['close'].rolling(20).mean()
    std = df['close'].rolling(20).std()
    df['bollinger_width'] = (2 * std) / mid

    # high-low range
    df['high_low_range'] = df['high'] - df['low']

    # z-score
    df['zscore_price'] = (df['close'] - mid) / (std + 1e-6)

    # 可扩展：RSI, volume/depth ratio等
    return df

def compute_long_freq_features(df_kline_daily):
    df = df_kline_daily.copy()
    for win in [30, 60, 180, 360]:
        df[f'ma_{win}d'] = df['close'].rolling(win).mean()
        df[f'volume_{win}d'] = df['volume'].rolling(win).sum()
        df[f'price_ma_diff_{win}d'] = (df['close'] - df[f'ma_{win}d']) / (df[f'ma_{win}d'] + 1e-6)
    
    # 示例：ATR
    df['tr'] = np.maximum(df['high'] - df['low'], np.abs(df['close'] - df['close'].shift(1)))
    df['atr_30d'] = df['tr'].rolling(30).mean()
    return df


In [20]:
grouped.loc[0,'bids']

array(['[111145.88, 0.49071]', '[111145.87, 0.00081]',
       '[111145.85, 0.00021]', '[111145.74, 0.00032]',
       '[111145.73, 0.32393]', '[111145.12, 0.00134]',
       '[111145.0, 0.001]', '[111144.82, 5e-05]', '[111144.8, 0.00018]',
       '[111144.55, 5e-05]'], dtype='<U20')

In [21]:
grouped.loc[0,'asks']

array(['[111145.89, 11.09448]', '[111145.9, 0.00035]',
       '[111145.91, 0.0001]', '[111145.92, 0.2158]',
       '[111145.93, 0.00035]', '[111146.51, 0.00134]',
       '[111146.77, 0.01817]', '[111147.54, 0.0001]',
       '[111147.55, 0.07137]', '[111147.61, 0.0001]'], dtype='<U21')

In [19]:
type(df['bids'][1])

list

In [8]:
features_df

,timestamp,spread,micro_price,imbalance
0,2025-07-10 16:20:42.758256,0.01,111145.889576,-0.915287
1,2025-07-10 16:20:42.776814,0.01,111145.889566,-0.913265
2,2025-07-10 16:20:42.886450,0.01,111145.889566,-0.913268
3,2025-07-10 16:20:42.986986,0.01,111145.889566,-0.913267
4,2025-07-10 16:20:43.286282,0.01,111145.889567,-0.913320
...,...,...,...,...
2498061,2025-07-13 13:58:01.454406,0.01,117765.356919,-0.383781
2498062,2025-07-13 13:58:01.555922,0.01,117765.356919,-0.383786
2498063,2025-07-13 13:58:01.654503,0.01,117765.356916,-0.383128
2498064,2025-07-13 13:58:01.758218,0.01,117765.356931,-0.386256


In [12]:
features_df.describe()

,spread,micro_price,imbalance
count,2.498066e+06,2.498066e+06,2.498066e+06
mean,1.276961e-02,1.162281e+05,-5.877526e-02
std,3.136831e-01,2.416247e+03,6.261700e-01
min,1.000000e-02,1.105000e+05,-9.999932e-01
25%,1.000000e-02,1.159249e+05,-6.418617e-01
50%,1.000000e-02,1.174420e+05,-8.206600e-02
75%,1.000000e-02,1.177412e+05,5.022533e-01
max,1.666100e+02,1.188652e+05,9.999976e-01


In [ ]:
interval_2 = "1h"
zip_folder_2 = "./1hkline" 

os.makedirs(zip_folder_2, exist_ok=True)

date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m-%d")
    filename = f"{symbol}-{interval_2}-{date_str}.zip"
    url = f"{base_url}/{symbol}/{interval_2}/{filename}"
    save_path = os.path.join(zip_folder_2, filename)
    
    try:
        print(f"Downloading {filename}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            #print(f"Saved: {filename}")
        else:
            print(f"Failed: {filename} (status code {response.status_code})")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")
    
    date += datetime.timedelta(days=1)
 

# 遍历当前目录所有 zip 文件
for filename in os.listdir(zip_folder_2):
    if filename.endswith(".zip"):
        zip_path = os.path.join(zip_folder_2, filename)
        extract_dir = os.path.join(zip_folder_2, filename.replace(".zip", ""))
        
        print(f"Extracting {filename} to {extract_dir} ...")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
        except zipfile.BadZipFile:
            print(f"Bad zip file: {filename}")